# Test profilage 

In [2]:
## Chargement des labels naf, rome, ogr
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# NAF
naf_labels = pd.read_csv('../ressources/list_NAF_LBB.csv', sep='|', encoding="utf-8")
naf_labels.columns = ['nafdot', 'naf', 'label']
print(f"Obtained {len(naf_labels)} NAF labels")
# display(HTML(naf_labels.head(5).to_html()))

# ROME
rome_labels = pd.read_csv('../ressources/liste_rome_LBB.csv', sep=',', encoding="utf-8")
rome_labels.columns = ['rome', 'rome_1', 'rome_2', 'rome_3', 'label', 'slug']
print(f"Obtained {len(rome_labels)} ROME labels")
# display(HTML(rome_labels.head(5).to_html()))

# Chargement des statistiques d'emploi
emploi_rome_naf = pd.read_csv('../ressources/contrats_30j.csv', sep=',', encoding="utf-8")[['ROME', 'APE700', 'nb_embauches']]
emploi_rome_naf.columns = ['rome', 'naf', 'embauches']
# display(HTML(emploi_rome_naf.head(5).to_html()))
        
    
# Calcul des ratios
naf_embauches = emploi_rome_naf[['naf', 'embauches']].groupby('naf').agg(
    embauches_total_n=pd.NamedAgg(column='embauches', aggfunc=sum)
)
rome_embauches = emploi_rome_naf[['rome', 'embauches']].groupby('rome').agg(
    embauches_total_r=pd.NamedAgg(column='embauches', aggfunc=sum)
)


naf_i = (
emploi_rome_naf
    .merge(naf_embauches, on='naf')
    .merge(rome_embauches, on='rome')
)
naf_i['ratio_naf'] = ( naf_i.embauches / naf_i.embauches_total_n ) * 100
naf_i['ratio_rome'] = ( naf_i.embauches / naf_i.embauches_total_r ) * 100

naf_i = naf_i.sort_values(by=['ratio_naf'], ascending=False)
naf_i.columns


Obtained 732 NAF labels
Obtained 531 ROME labels


Index(['rome', 'naf', 'embauches', 'embauches_total_n', 'embauches_total_r',
       'ratio_naf', 'ratio_rome'],
      dtype='object')

In [12]:
emploi_rome_naf.sort_values(by=['embauches'], ascending=False)

,rome,naf,embauches
131860,K2503,8010Z,67797
67564,G1803,5610A,48196
63678,G1602,5610A,47564
128140,K2204,8121Z,43282
55321,F1703,4399C,33900
...,...,...,...
47187,F1402,2222Z,1
47186,F1402,2219Z,1
121857,K1802,1084Z,1
47185,F1402,2051Z,1


In [5]:
rome_labels
rome_labels.loc[rome_labels.rome=='N4301'].index[0].astype(np.int32)
rome_labels

,rome,rome_1,rome_2,rome_3,label,slug
0,A1101,A,11,1,Conduite d'engins agricoles et forestiers,conduite-d-engins-agricoles-et-forestiers
1,A1201,A,12,1,Bûcheronnage et élagage,bucheronnage-et-elagage
2,A1202,A,12,2,Entretien des espaces naturels,entretien-des-espaces-naturels
3,A1203,A,12,3,Entretien des espaces verts,entretien-des-espaces-verts
4,A1204,A,12,4,Protection du patrimoine naturel,protection-du-patrimoine-naturel
...,...,...,...,...,...,...
526,N4301,N,43,1,Conduite sur rails,conduite-sur-rails
527,N4302,N,43,2,Contrôle des transports en commun,controle-des-transports-en-commun
528,N4401,N,44,1,Circulation du réseau ferré,circulation-du-reseau-ferre
529,N4402,N,44,2,Exploitation et manoeuvre des remontées mécani...,exploitation-et-manoeuvre-des-remontees-mecani...


In [6]:
naf_i

,rome,naf,embauches,embauches_total_n,embauches_total_r,ratio_naf,ratio_rome
122348,N1103,0116Z,3,3,155192,100.000000,0.001933
5147,A1405,0127Z,2,2,8202,100.000000,0.024384
48151,G1204,0510Z,1,1,44723,100.000000,0.002236
1601,A1203,0721Z,2,2,68967,100.000000,0.002900
15857,D1202,9602A,18296,23888,36778,76.590757,49.747131
...,...,...,...,...,...,...,...
160730,H3403,5610A,1,268884,223,0.000372,0.448430
163191,K1503,5610A,1,268884,47,0.000372,2.127660
167387,C1106,5610A,1,268884,206,0.000372,0.485437
162840,H2405,5610A,1,268884,49,0.000372,2.040816


In [7]:
np.zeros(rome_labels.size)

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
df = naf_i.loc[naf_i.naf == '5621Z'].loc[naf_i.ratio_naf.ge(5)]

# np.set_printoptions(threshold=np.inf)

ls = {}
for ni, nr in naf_labels.iterrows():
    df = naf_i.loc[naf_i.naf == nr.naf].loc[naf_i.ratio_naf.ge(2)]
    vec = np.zeros(rome_labels.size)
    for index, row in df.iterrows():
        rome_i = rome_labels.loc[rome_labels.rome==row['rome']].index[0].astype(np.int32)
        vec[rome_i] = 1
    ls[nr.naf] = vec


In [18]:
search_romes = ['G1603','K2204', 'N4105', 'G1605']
search_romes = ['K2204',
'H3302',
'K2204',
'F1705',
'I1603',
'H2909',
'F1602',
'F1705',
'K2303',
'N4101',
'F1704',
'F1704',
'F1702',
'N1103',
'F1613',
'F1608',
'F1606',
'G1603',
'F1503',
'F1503',
'F1704',
'N1105',
'F1604',
'F1502',
'N1101',
'D1507',
'N4105',
'F1704',
'F1302',
'F1701']

search_vec = np.zeros(rome_labels.size)
for rome in search_romes:
    rome_i = rome_labels.loc[rome_labels.rome==rome].index[0].astype(np.int32)
    search_vec[rome_i] = 1
    print(f'rome : {rome_labels.loc[rome_labels.rome==rome].label})')
cols = ['naf', 'score', 'label']
res = pd.DataFrame([], columns=cols)
for naf, vec in ls.items():
    label = naf_labels.loc[(naf_labels['naf'] == naf, 'label')].iloc[0]
    dfm = pd.DataFrame([[naf,  np.linalg.norm(vec-search_vec), label]], columns=cols)
    res = res.append(dfm)


rome : 414    Nettoyage de locaux
Name: label, dtype: object)
rome : 292    Opérations manuelles d'assemblage, tri ou emba...
Name: label, dtype: object)
rome : 414    Nettoyage de locaux
Name: label, dtype: object)
rome : 172    Pose de canalisations
Name: label, dtype: object)
rome : 321    Maintenance d'engins de chantier, levage, manu...
Name: label, dtype: object)
rome : 280    Montage-assemblage mécanique
Name: label, dtype: object)
rome : 156    Électricité bâtiment
Name: label, dtype: object)
rome : 172    Pose de canalisations
Name: label, dtype: object)
rome : 417    Nettoyage des espaces urbains
Name: label, dtype: object)
rome : 517    Conduite de transport de marchandises sur long...
Name: label, dtype: object)
rome : 171    Préparation du gros oeuvre et des travaux publics
Name: label, dtype: object)
rome : 171    Préparation du gros oeuvre et des travaux publics
Name: label, dtype: object)
rome : 169    Construction de routes et voies
Name: label, dtype: object)
rome : 4

In [19]:
res.sort_values(by=['score'], ascending=True).head(10)

,naf,score,label
0,3821Z,3.872983,Traitement et élimination des déchets non dang...
0,3811Z,4.000000,Collecte des déchets non dangereux
0,3832Z,4.000000,Récupération de déchets triés
0,4631Z,4.123106,Commerce de gros (commerce interentreprises) d...
0,4311Z,4.123106,Travaux de démolition
0,5210A,4.123106,Entreposage et stockage frigorifique
0,2399Z,4.242641,Fabrication d'autres produits minéraux non mét...
0,3900Z,4.242641,Dépollution et autres services de gestion des ...
0,5224B,4.242641,Manutention non portuaire
0,4633Z,4.358899,Commerce de gros (commerce interentreprises) d...
